In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np

#helper function to get responses of the api and store each layer in a dictionary
def get_responses(*args):
    list_of_responses={}
    for parm in args:
        url = f"https://emotional.byteroad.net/collections/{parm}/items?f=json&lang=en-US&limit=10000&skipGeometry=false&offset=0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data_dict = json.loads(response.content)
            list_of_responses[parm]=data_dict
        else:
            print("Error: API request unsuccessful.")
    return list_of_responses

In [2]:
from branca.element import MacroElement
from jinja2 import Template

class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('layeradd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('layerremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)    

In [3]:
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl
import branca.colormap as cm
import branca
import numpy as np

#dynamic legend values
column_names = ['mean_NO2_Gade', 'mean_PM25_Gade', 'mean_PM10_Gade','noise']


responses = [
    get_responses('air_pollution_wzones')['air_pollution_wzones']['features'],
    get_responses('noise_wzones')['noise_wzones']['features']
]

data = {column_name: [] for column_name in column_names}

for response_data_index, response_data in enumerate(responses):
    for feature in response_data:
        properties = feature['properties']
        if response_data_index == 0:
            # Process the first response (air pollution data)
            for column_name in column_names[:3]:
                value = properties.get(column_name, None)
                data[column_name].append(value)
        else:
            # Process the second response (noise data)
            value = properties.get('noise', None)  # Assuming 'noise' is the column name
            data['noise'].append(value)

df = pd.DataFrame(data)

# Convert columns to numeric
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

min_values = df.min(skipna=True)
max_values = df.max(skipna=True)

num_groups = 4
group_boundaries = {}
for column in df.columns:
    min_value = min_values[column]
    max_value = max_values[column]
    
    if min_value is None or max_value is None:
        continue  # Skip columns with missing or null values
        
    group_size = (max_value - min_value) / num_groups
    boundaries = np.arange(min_value, max_value + group_size, group_size)
    group_boundaries[column] = boundaries

# Print the group boundaries for each column
for column, boundaries in group_boundaries.items():
    print(f"Group boundaries for column '{column}':")
    print(boundaries)
    print()

Group boundaries for column 'mean_NO2_Gade':
[11.05954545 14.58167365 18.10380185 21.62593005 25.14805825 28.67018645]

Group boundaries for column 'mean_PM25_Gade':
[ 9.78675435 10.22210743 10.65746051 11.09281359 11.52816667]

Group boundaries for column 'mean_PM10_Gade':
[15.1520339  16.24515042 17.33826695 18.43138347 19.5245    ]

Group boundaries for column 'noise':
[0.  1.5 3.  4.5 6. ]



In [25]:
def hexmap(responses, column_name, color, feature_group):
    for response in responses:
        for feature in response:
            properties = feature['properties']
            if column_name in properties:
                value = float(properties[column_name])
                # Format the value to display two decimal places
                formatted_value = "{:.2f}".format(value)
                popup_content = folium.Popup(f"Mean Value: {formatted_value}", parse_html=True)
                geoj = folium.GeoJson(feature, style_function=color, popup=popup_content)
                feature_group.add_child(geoj)

#No2

color_no2 = lambda x: {
    'fillColor': '#FFFFFF' if float(x['properties']['mean_NO2_Gade']) < 11.06 else 
    '#E6F5FF' if 11.06 <= float(x['properties']['mean_NO2_Gade']) < 14.58 else 
    '#B2D4FF' if 14.58 <= float(x['properties']['mean_NO2_Gade']) < 18.10 else 
    '#8A7FFF' if 18.10 <= float(x['properties']['mean_NO2_Gade']) < 21.63 else
    '#9370DB',
    'color': 'white',
    'weight': 1,
    'fillOpacity': '0.7'
}

fg1 = folium.FeatureGroup(name='Nitrogen Dioxide (NO2)', overlay=True, control=False)
hexmap(responses, 'mean_NO2_Gade', color_no2, fg1)
scale_fg1 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB'], index=[11.06,14.58,18.10,21.63,28.67], vmin=11.06, vmax=28.67, caption='NO2 (μg/m³)')


#PM25

color_pm25 = lambda x: {
    'fillColor': '#FFFFFF' if float(x['properties']['mean_PM25_Gade']) < 9.79 else 
    '#E6F5FF' if 9.79 <= float(x['properties']['mean_PM25_Gade']) < 10.22 else 
    '#B2D4FF' if 10.22 <= float(x['properties']['mean_PM25_Gade']) < 10.66 else 
    '#8A7FFF' if 10.66 <= float(x['properties']['mean_PM25_Gade']) < 11.09 else
    '#9370DB' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg2 = folium.FeatureGroup(name = 'Particulate Matter (PM2.5)', overlay=True, control=False)
hexmap(responses, 'mean_PM25_Gade', color_pm25, fg2)
scale_fg2 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB'],index=[9.79,10.22,10.66,11.09,11.53], vmin=9.79, vmax=11.53, caption='Particulate Matter (PM25)')

#PM10 

color_pm10 = lambda x: {
    'fillColor': '#FFFFFF' if float(x['properties']['mean_PM10_Gade']) < 15.15 else 
    '#E6F5FF' if 15.15 <= float(x['properties']['mean_PM10_Gade']) < 16.25 else
    '#B2D4FF' if 16.25 <= float(x['properties']['mean_PM10_Gade']) < 17.34 else
    '#8A7FFF' if 17.34 <= float(x['properties']['mean_PM10_Gade']) < 18.43 else
    '#9370DB', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}
fg3 = folium.FeatureGroup(name='Particulate Matter (PM10)',overlay=True, control=False)
hexmap(responses, 'mean_PM10_Gade',color_pm10, fg3)
scale_fg3 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB'], index=[15.15,16.25,17.34,18.43,19.52], vmin=15.15, vmax=19.52, caption='PM10 (μg/m³)')

#Noise

color_noise = lambda x: {
    'fillColor': '#FFFFFF' if float(x['properties']['noise']) < 0 else
    '#FFFFCC' if  0 <= float(x['properties']['noise']) < 1.5 else             
    '#FFCC99' if 1.5 <= float(x['properties']['noise']) < 3 else
    '#FF9933' if 3 <= float(x['properties']['noise']) < 4.5 else
    '#FF6633', 'color': 'white','weight': 1,'fillOpacity': '0.7'}
fg4 = folium.FeatureGroup(name='Mean of the total noise levels for each noise point within that particular zone',overlay=True, control=False)
hexmap(responses, 'noise', color_noise, fg4)
scale_fg4 = branca.colormap.StepColormap(['#FFFFCC','#FFCC99','#FF9933','#FF6633'], index=[0,1.5,3,4.5,6], vmin=0, vmax=6, caption='Noise Levels')

In [26]:
# Add each layer to the map and save it as a separate HTML file

for i in range(1, 5):
    # Create a new map object for each iteration
    m = folium.Map(location=[55.707945, 12.568337], control_scale=True, zoom_start=11, tiles=None)
    
    base_map1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map1)
    # Add satellite imagery basemap
    base_map2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ).add_to(base_map2)

    base_map1.add_to(m)
    base_map2.add_to(m)
    
    fg_name = "fg" + str(i)
    fg = globals()[fg_name]
    m.add_child(fg)
    
    scale_name = "scale_fg" + str(i)
    scale = globals()[scale_name]
    m.add_child(scale)
    m.add_child(BindColormap(fg, scale))

    # Add LayerControl to toggle between basemaps
    folium.LayerControl(position='bottomleft').add_to(m)
    
    map_fname = f'Output_Copenhagen_Layer{i}.html'
    m.save(map_fname)